In [10]:
import pandas as pd
import numpy as np
import networkx as nx
import plotly.express as px
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler

def data_report(file_name):
    fpath='/home/adityapandey/Downloads/project-sem5/Data Sets/'
    from ydata_profiling import ProfileReport
    df = pd.read_csv(fpath+file_name)
    profile = ProfileReport(df, title="final report1", explorative=True)
    profile.to_file("final report1.html")
    


In [11]:


def load_data():
    fpath='/home/adityapandey/Downloads/project-sem5/Data Sets/'
    indian_students = pd.read_csv(fpath+"IndianStudentsAbroad.csv")
    cost_living = pd.read_csv(fpath+"Cost_of_Living_Index_by_Country_2024.csv")
    tuition = pd.read_csv(fpath+"International_Education_Costs.csv")
    reputation = pd.read_csv(fpath+"QS World University Rankings 2025 (Top global universities).csv",encoding='latin1')
    return indian_students, cost_living, tuition, reputation

indian_students, cost_living, tuition, reputation = load_data()
indian_students.drop(columns=['index'], inplace=True)
print(indian_students.head())



      Country  No of Indian Students  Percentage  Unnamed: 3
0     Armenia                    900    0.201434         NaN
1   Australia                  66886   14.970109   52.105095
2     Austria                    300    0.067145         NaN
3  Azerbaijan                     11    0.002462         NaN
4     Bahrain                  27000    6.043013   76.500961


In [ ]:
indian_students.loc[indian_students['Country']=='US(2015-16)','Country']='United States'
indian_students.loc[indian_students['Country']=='UK(2015-16)','Country']='United Kingdom'
print(indian_students)

                        Country  No of Indian Students  Percentage  Unnamed: 3
0                       Armenia                    900    0.201434         NaN
1                     Australia                  66886   14.970109   52.105095
2                       Austria                    300    0.067145         NaN
3                    Azerbaijan                     11    0.002462         NaN
4                       Bahrain                  27000    6.043013   76.500961
..                          ...                    ...         ...         ...
70               United Kingdom                  16715    3.741073         NaN
71                      Ukraine                   7963    1.782241         NaN
72  United Arab Emirates(Dubai)                   5500    1.230984         NaN
73                United States                 165918   37.134985         NaN
74                   Uzbekistan                     50    0.011191         NaN

[75 rows x 4 columns]


In [13]:
tuition.columns = tuition.columns.str.strip().str.lower().str.replace(" ", "_")
tuition.loc[tuition['country']=='USA','country']='United States'
tuition.loc[tuition['country']=='UK','country']='United Kingdom'
print(tuition.head())
tuition["total_cost"] = (tuition["tuition_usd"] * tuition["duration_years"]) + \
                   (tuition["rent_usd"] *tuition["duration_years"]) + \
                tuition["visa_fee_usd"] +tuition["insurance_usd"]
"""nums_col2=['exchange_rate','living_cost_index','tuition_usd','duration_years','rent_usd','visa_fee_usd','insurance_usd','total_cost']  
tuition[nums_col2]=scaler.fit_transform(tuition[nums_col2])""" 



   country         city                            university  \
0  Algeria      Algiers  University of Science and Technology   
1  Algeria         Oran                    University of Oran   
2  Algeria  Constantine             University of Constantine   
3  Algeria       Annaba              Badji Mokhtar University   
4  Algeria      Tlemcen                 University of Tlemcen   

                program     level  duration_years  tuition_usd  \
0      Computer Science  Bachelor             3.0         1200   
1  Computer Engineering    Master             2.0         1100   
2        Data Analytics       PhD             4.0         1300   
3  Computer Engineering    Master             2.0         1000   
4   Information Systems  Bachelor             3.0          900   

   living_cost_index  rent_usd  visa_fee_usd  insurance_usd  exchange_rate  
0               38.5       250            80            200          137.5  
1               36.4       220            80            20

"nums_col2=['exchange_rate','living_cost_index','tuition_usd','duration_years','rent_usd','visa_fee_usd','insurance_usd','total_cost']  \ntuition[nums_col2]=scaler.fit_transform(tuition[nums_col2])"

In [ ]:
cost_living.columns = cost_living.columns.str.strip().str.lower().str.replace(" ", "_")
print(cost_living.head())


   rank      country  cost_of_living_index  rent_index  \
0     1  Switzerland                 101.1        46.5   
1     2      Bahamas                  85.0        36.7   
2     3      Iceland                  83.0        39.2   
3     4    Singapore                  76.7        67.2   
4     5     Barbados                  76.6        19.0   

   cost_of_living_plus_rent_index  groceries_index  restaurant_price_index  \
0                            74.9            109.1                    97.0   
1                            61.8             81.6                    83.3   
2                            62.0             88.4                    86.8   
3                            72.1             74.6                    50.4   
4                            48.9             80.8                    69.4   

   local_purchasing_power_index  
0                         158.7  
1                          54.6  
2                         120.3  
3                         111.1  
4           

In [ ]:

reputation.columns = reputation.columns.str.strip()
print(reputation.head())
print(reputation.columns.tolist())




  RANK_2025 RANK_2024                              Institution_Name  \
0         1         1  Massachusetts Institute of Technology (MIT)    
1         2         6                       Imperial College London   
2         3         3                          University of Oxford   
3         4         4                            Harvard University   
4         5         2                       University of Cambridge   

         Location    Region SIZE FOCUS RES. STATUS  Academic_Reputation_Score  \
0   United States  Americas    M    CO   VH      B                      100.0   
1  United Kingdom    Europe    L    FC   VH      A                       98.5   
2  United Kingdom    Europe    L    FC   VH      A                      100.0   
3   United States  Americas    L    FC   VH      B                      100.0   
4  United Kingdom    Europe    L    FC   VH      A                      100.0   

   ... International_Faculty_Rank  International_Students_Score  \
0  ...             

In [ ]:
reputation.columns = reputation.columns.str.strip().str.lower().str.replace(" ", "_")
indian_students.columns = indian_students.columns.str.strip().str.lower().str.replace(" ", "_")
df = tuition.merge(cost_living, on="country", how="left") \
            .merge(indian_students, on="country", how="left") \
            .merge(reputation, left_on="university", right_on="institution_name", how="left")
# Example: Drop rows where 'rank' is NaN (missing)
df_cleaned = df.dropna(subset=['rank_2025'])
df_cleaned = df_cleaned.drop(df_cleaned.loc[df_cleaned['country'] == 'UAE'].index)
df_cleaned = df_cleaned.drop(df_cleaned.loc[df_cleaned['country'] == 'South Korea'].index)
df_cleaned=df_cleaned.drop(columns=['institution_name','location'])
print(df_cleaned.head())
df_cleaned=df_cleaned.drop(columns=['unnamed:_3','rank_2024','region','size','focus','res.','status'])
fpath='/home/adityapandey/Downloads/project-sem5/Data Sets/'
df_cleaned.to_csv(fpath+"merged_indian_students_abroad.csv", index=False)



      country        city                university                  program  \
16  Australia  Gold Coast       Griffith University   Information Technology   
17  Australia      Hobart    University of Tasmania             Data Science   
19  Australia  Wollongong  University of Wollongong  Artificial Intelligence   
21  Australia  Townsville     James Cook University           Data Analytics   
39  Australia   Melbourne         Monash University         Computer Science   

       level  duration_years  tuition_usd  living_cost_index  rent_usd  \
16  Bachelor             3.0        31200               67.5      1100   
17    Master             2.0        30800               65.4       950   
19    Master             2.0        32900               67.2      1050   
21    Master             2.0        31200               65.8       950   
39  Bachelor             3.0        39800               72.8      1700   

    visa_fee_usd  ...  international_faculty_rank  \
16           450  ...